In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras import models, Model
from tensorflow.keras.initializers import RandomNormal
import qkeras
from qkeras import *
from sparsepixels.layers import *
from sparsepixels.utils import *
import h5py
import glob
import os

In [ ]:
def build_dummy():
    quantizer = quantized_bits(6, 0, alpha=1)

    x_in = keras.Input(shape=(10, 20, 1), name='x_in')
    x = QConv2D(filters=2, kernel_size=(3, 3), use_bias=True, name='conv1',
                padding='same', strides=1,
                kernel_quantizer=quantizer, bias_quantizer=quantizer,
                kernel_initializer=RandomNormal(mean=0, stddev=1),
                bias_initializer=RandomNormal(mean=0, stddev=1))(x_in)

    return keras.Model(x_in, x, name='dummy')

model_dummy = build_dummy()
model_dummy.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                    loss='categorical_crossentropy', metrics = ['accuracy'])
print(model_dummy.summary())

In [ ]:
import hls4ml
config = hls4ml.utils.config_from_keras_model(model_dummy, granularity='name', backend='Vitis')
config

model_dummy_hls = hls4ml.converters.convert_from_keras_model(
    model_dummy,
    hls_config=config,
    project_name='hls_dummy',
    backend='Vitis',
    output_dir='hls_proj/scaling/dummy',
    part='xcu250-figd2104-2L-e',
    #io_type='io_stream',
    io_type='io_parallel',
)

model_dummy_hls.compile()
model_dummy_hls.write()

In [ ]:
import numpy as np
numbers = np.random.rand(100) * 2
print(' '.join(f'{x:.4g}' for x in numbers))